# 多クラス分類モデルの評価
- scikit-learnを用いたmicro平均、macro平均の求め方を確認しよう

## 1. ライブラリの読み込み

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np

# データセットを読み込む関数
from sklearn.datasets import load_iris

# モデルを表すクラス
from sklearn.linear_model import SGDClassifier

# 分類問題における評価指標の関数
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix
from sklearn.metrics import classification_report, balanced_accuracy_score

## 2. データの読み込み
- [The Iris Dataset](https://scikit-learn.org/stable/auto_examples/datasets/plot_iris_dataset.html)
  - アヤメの品種分類のデータセット
  - 目的変数（クラス）
      - Setosa
      - Versicolor
      - Virginica
  - 説明変数
      - Sepal Length（萼片の長さ）
      - Sepal Width（萼片の幅）
      - Petal Length（花弁の長さ）
      - Petal Width（花弁の幅）

In [2]:
# アヤメの品種（Setosa, Versicolor, Virginica）を分類する3クラス分類問題
iris = load_iris()

# 目的変数
y = iris.target
# 説明変数（萼片の長さ、萼片の幅、花弁の長さ、花弁の幅）
X = iris.data 

# ラベルに対応する文字列のリスト
label_txt = iris.target_names

## 3. モデルの構築・学習

In [3]:
# ロジスティック回帰モデルの構築
clf = SGDClassifier(loss='log_loss', max_iter=10000, fit_intercept=True, 
                    random_state=1234, tol=1e-3)

# 学習を実行
clf.fit(X, y)

SGDClassifier(loss='log_loss', max_iter=10000, random_state=1234)

## 4.混同行列の確認 

In [4]:
# ラベルを予測
y_pred = clf.predict(X)


# 予測値と正解のクロス集計（混同行列）
conf_mat = confusion_matrix(y, y_pred)
conf_mat = pd.DataFrame(conf_mat, 
                        index=label_txt, 
                        columns=label_txt)
conf_mat

,setosa,versicolor,virginica
setosa,50,0,0
versicolor,41,4,5
virginica,0,0,50


## 5. Micro 平均
- 全クラスのデータをまとめて扱い、評価指標を求める
    - Precision・Recall・F1は、Accuracyと同じ値になるため、計算する意味はない

In [5]:
# 正解率を計算
accuracy =  accuracy_score(y, y_pred)
print('正解率（Accuracy） = {:.3f}%'.format(100 * accuracy))

正解率（Accuracy） = 69.333%


## 6. Macro 平均
- クラスごとに評価指標を計算し、その平均を求める
    - [classification_report()](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html)を用いると、簡単に計算できる
    - supportは、正解データにおけるクラスごとのデータの個数
    - weighted averageは、supportの値を重みとした重みつき平均

In [6]:
# Presicion, Recall, F1, Accuracyをまとめて表示
scores = classification_report(y, y_pred, target_names=label_txt)
print(scores)

              precision    recall  f1-score   support

      setosa       0.55      1.00      0.71        50
  versicolor       1.00      0.08      0.15        50
   virginica       0.91      1.00      0.95        50

    accuracy                           0.69       150
   macro avg       0.82      0.69      0.60       150
weighted avg       0.82      0.69      0.60       150



## 7. Macro平均（手計算で行う場合）
- macro_accuracyは [balanced_accuracy_score()](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.balanced_accuracy_score.html) を用いて計算できる
    - 今回はクラスごとのデータサイズに偏りがないため、Micro平均と同じ値になる
- それ以外の指標は以下の手順で計算
    - [precision_recall_fscore_support()](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_recall_fscore_support.html) で各クラスの指標を求める
    - 求めた値の平均を計算



In [7]:
# macro-Accuracyの計算
macro_accuracy = balanced_accuracy_score(y, y_pred)
print('macro-Accuracy = {:.3f}%'.format(100 * macro_accuracy))

# 各クラスのPrecision, Recall, F1を計算
precision, recall, f1_score, _ = precision_recall_fscore_support(y, y_pred)

# macro-Precisionの計算
macro_precision = np.mean(precision)
print('macro-Precision = {:.3f}%'.format(100 * macro_precision))

# macro-Recallの計算
macro_recall = np.mean(recall)
print('macro-Recall = {:.3f}%'.format(100 * macro_recall))

# macro-f1の計算①(各クラスのF1を平均する)
macro_f1 = np.mean(f1_score)
print('macro-F1① = {:.3f}%'.format(100 * macro_f1))

# macro-f1の計算②（macro-Recallとmacro-Precisionを用いて計算）
macro_f1 = 2*macro_precision*macro_recall/(macro_precision+macro_recall)
print('macro-F1② = {:.3f}%'.format(100 * macro_f1))

macro-Accuracy = 69.333%
macro-Precision = 81.951%
macro-Recall = 69.333%
macro-F1① = 60.325%
macro-F1② = 75.116%
